In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
import joblib

c:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Load the dataset
df = pd.read_csv('AirlineData.csv')

In [3]:
df.head()


,Airline,Source,Destination,Number of Stops,Class,Date,Total_Stopover_Time,price in CAD,days_left,Departure_24hr,Arrival_24hr,Arrival_Day_Offset
0,Etihad Airways,Toronto Pearson Intl,Bengaluru Intl,1,Economy Class,2024-06-02,130,2340,1,22:10,3:05,2
1,Delta,Toronto Pearson Intl,Bengaluru Intl,1,Economy Class,2024-06-02,1335,1347,1,22:40,23:15,2
2,Multiple Airlines,Toronto Pearson Intl,Bengaluru Intl,2,Economy Class,2024-06-02,420,1934,1,21:20,7:45,2
3,Multiple Airlines,Toronto Pearson Intl,Bengaluru Intl,2,Economy Class,2024-06-02,380,2291,1,21:20,6:50,2
4,Multiple Airlines,Toronto Pearson Intl,Bengaluru Intl,2,Economy Class,2024-06-02,175,2661,1,23:40,6:25,2


In [4]:
# Preprocess the data
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
df['DayOfWeek'] = df['Date'].dt.dayofweek

def time_to_minutes(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

df['Departure_Minutes'] = df['Departure_24hr'].apply(time_to_minutes)
df['Arrival_Minutes'] = df['Arrival_24hr'].apply(time_to_minutes)

In [5]:
# Create and store label encoders
le_dict = {}
categorical_columns = ['Airline', 'Source', 'Destination', 'Class']
for col in categorical_columns:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col])
    le_dict[col] = le

# Save label encoders
joblib.dump(le_dict, 'label_encoders.joblib')

['label_encoders.joblib']

In [6]:
df

,Airline,Source,Destination,Number of Stops,Class,Date,Total_Stopover_Time,price in CAD,days_left,Departure_24hr,Arrival_24hr,Arrival_Day_Offset,Month,DayOfWeek,Departure_Minutes,Arrival_Minutes,Airline_encoded,Source_encoded,Destination_encoded,Class_encoded
0,Etihad Airways,Toronto Pearson Intl,Bengaluru Intl,1,Economy Class,2024-06-02,130,2340,1,22:10,3:05,2,6,6,1330,185,20,14,1,1
1,Delta,Toronto Pearson Intl,Bengaluru Intl,1,Economy Class,2024-06-02,1335,1347,1,22:40,23:15,2,6,6,1360,1395,15,14,1,1
2,Multiple Airlines,Toronto Pearson Intl,Bengaluru Intl,2,Economy Class,2024-06-02,420,1934,1,21:20,7:45,2,6,6,1280,465,30,14,1,1
3,Multiple Airlines,Toronto Pearson Intl,Bengaluru Intl,2,Economy Class,2024-06-02,380,2291,1,21:20,6:50,2,6,6,1280,410,30,14,1,1
4,Multiple Airlines,Toronto Pearson Intl,Bengaluru Intl,2,Economy Class,2024-06-02,175,2661,1,23:40,6:25,2,6,6,1420,385,30,14,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824014,Multiple Airlines,Kano Aminu Kano Intl Apt,Toronto Pearson Intl,2,Business Class,2024-08-30,555,12394,49,13:35,15:30,1,8,4,815,930,30,5,14,0
824015,Multiple Airlines,Kano Aminu Kano Intl Apt,Toronto Pearson Intl,3,Business Class,2024-08-30,805,12269,49,10:45,19:55,1,8,4,645,1195,30,5,14,0
824016,Multiple Airlines,Kano Aminu Kano Intl Apt,Toronto Pearson Intl,3,Business Class,2024-08-30,805,12407,49,10:45,19:55,1,8,4,645,1195,30,5,14,0
824017,Multiple Airlines,Kano Aminu Kano Intl Apt,Toronto Pearson Intl,3,Business Class,2024-08-30,530,12001,49,13:35,17:20,1,8,4,815,1040,30,5,14,0


In [38]:
# Define features
features = ['Number of Stops', 
            'Month', 'DayOfWeek', 'Departure_Minutes', 'Arrival_Minutes',
            'Airline_encoded', 'Source_encoded', 'Destination_encoded', 'Class_encoded']

X = df[features]
y = df['price in CAD']

In [42]:
# Split the data and train the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = ExtraTreesRegressor(bootstrap=False, max_features=0.7500000000000001, min_samples_leaf=1, min_samples_split=4, n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

ExtraTreesRegressor(max_features=0.7500000000000001, min_samples_split=4,
                    n_jobs=-1, random_state=42)

In [44]:
# Save the model
joblib.dump(model, 'extratrees_model.joblib')

print("Preprocessing complete. Label encoders and model saved.")

Preprocessing complete. Label encoders and model saved.
